In [1]:
# 사건사고 Link, titles, Date, Name, Reporter, Contents(성공) 

import requests as rq
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import re

all_links = []
all_titles = []
all_Date = []
all_Name = []
all_Reporter = []
all_Contents = []

date_range = pd.date_range(start="2023-08-30", end="2024-08-30").strftime('%Y%m%d')

for date in tqdm(date_range, desc="Processing_1"):
    for page in range(1, 101):
        res = rq.get(f'https://news.daum.net/breakingnews/society/affair?page={page}&regDate={date}', headers={
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
        })
        soup = BeautifulSoup(res.text, 'html.parser')
        titles = soup.select('#mArticle .box_etc strong > a')
        Names = soup.select('span.info_news')

        if not titles:
            break

        for i in range(len(titles)):
            all_Date.append(date)
            all_links.append(titles[i]['href'])
            all_titles.append(titles[i].text)
            all_Name.append(Names[i].contents[0])

def fetch_reporter_and_content(link):
    res_2 = rq.get(link, headers={
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
    })
    soup = BeautifulSoup(res_2.text, 'html.parser')

    Reporter = soup.select('div.info_view > .txt_info')
    reporter_name = Reporter[0].text if Reporter else None

    contents = soup.select('#mArticle > div.news_view.fs_type1')
    content_text = re.sub(r'\s+', ' ', contents[0].text.replace("\n", "").strip()) if contents else ""

    return reporter_name, content_text

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(fetch_reporter_and_content, link): link for link in all_links}

    for future in tqdm(futures, desc="Processing_2"):
        try:
            reporter, content = future.result()
            all_Reporter.append(reporter)
            all_Contents.append(content)
        except:
            pass


Processing_2:  37%|███▋      | 130592/351323 [8:26:37<6:29:50,  9.44it/s]     

In [ ]:
df = pd.DataFrame({
    'Date' : all_Date,
    'Name' : all_Name,
    'Reporter' : all_Reporter,
    'Title': all_titles,
    'Content' : all_Contents,
    'Link': all_links
})

df

,Date,Name,Reporter,Title,Content,Link
0,20230830,연합뉴스,천정인,여수 만흥동서 빗길 교통사고…30대 운전자 숨져,119 구급차·응급환자 병원 이송 (PG) [정연주 제작] 일러스트 (여수=연합뉴스...,https://v.daum.net/v/20230830235616110
1,20230830,서울경제,안유진 인턴기자,"""제2·3의 혜빈이 나와도 이럴 거냐""···분당 흉기난동 피해자 母 '분노'",[서울경제] 김혜빈씨는 ‘분당 흉기 난동’ 사건의 피의자 최원종(22)이 몰던 차량...,https://v.daum.net/v/20230830234257951
2,20230830,서울경제,김태원 기자,"모텔서 2개월 여아 숨졌는데···국과수 ""외상 흔적 없다”",연합뉴스 [서울경제] 인천의 한 모텔에서 20대 부모와 함께 생활하던 생후 2개월 ...,https://v.daum.net/v/20230830231653793
3,20230830,매일경제,김대영 매경닷컴 기자(kdy7118@mk.co.kr),택시 태워준다면서 양손으로…대낮 버스정류장서 10대 여학생 강제추행,[사진 출처 = 연합뉴스] 술에 취한 상태로 버스를 기다리다 10대 여성들에게 택시...,https://v.daum.net/v/20230830225700701
4,20230830,서울경제,김태원 기자,여자친구·여동생까지 함께 ‘나락’으로···40억어치 마약 밀수조직 잡혔다,30일 인천지검 대회의실에서 문행용 인천공항본부세관 조사국장이 합동수사 결과를 발표...,https://v.daum.net/v/20230830225629698
...,...,...,...,...,...,...
1820,20230831,중앙일보,이수민,"""강아지 XX패고 화장실 가둬""…학대 글에 누리꾼도 화났다",지난 26일 익명 커뮤니티 '디시인사이드'에 올라온 강아지 사진. 견주 A씨는 강아...,https://v.daum.net/v/20230831000203215
1821,20230831,중앙일보,김민중,경찰 “스쿨존 속도제한 완화” 하루만에 번복,경찰이 다음 달 1일부터 심야시간대 스쿨존(어린이보호구역) 속도제한을 완화하겠다고 ...,https://v.daum.net/v/20230831000125198
1822,20230831,더팩트,최의종,"'저위험 권총' 도입 추진…""묻지마 범죄 근본 대책 아냐""","현장 '활용도', 면책 제도 관건…""장비 문제 아냐"" 지적도 경찰청이 지난 29일 ...",https://v.daum.net/v/20230831000043174
1823,20230831,아이뉴스24,김동현,"[결혼과 이혼] '딩크족' 바라던 아내, 자식 원하는 남편에 '이혼소송'",[아이뉴스24 김동현 기자] 자식 문제로 갈등을 겪다 이혼 위기에 놓인 부부 이야기...,https://v.daum.net/v/20230831000021161


In [ ]:
# 스티커 드디어 완성 (성공)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import re

all_Sticker = []

def process_link(link):
    driver = webdriver.Chrome()
    driver.get(link)
    try:
        sticker_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="alex_action_emotion"]'))
        )
        recommend_text = re.sub(r"(\d)([가-힣])", r"\1 \2", sticker_element.text.replace("\n", ""))
        return recommend_text
    
    except:
        pass

    finally:
        driver.quit()

with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(process_link, link) for link in all_links]
    
    for future in tqdm(futures, desc="Processing"):
        try:
            result = future.result()
            if result:
                all_Sticker.append(result)
        except:
            pass

In [8]:
#%pip install requests
#%pip install beautifulsoup4
#%pip install tqdm
#%pip install pandas

  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl (10.8 MB)
Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.
